In [1]:
!pip install -q transformers accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00


In [5]:
# https://drive.google.com/file/d/133u-TYuIU9DfUAy7qcnOtqEQJJbLhBAX/view?usp=sharing
!gdown 133u-TYuIU9DfUAy7qcnOtqEQJJbLhBAX

Downloading...
From: https://drive.google.com/uc?id=133u-TYuIU9DfUAy7qcnOtqEQJJbLhBAX
To: /content/shopee_reviews.csv
100% 929k/929k [00:00<00:00, 107MB/s]


In [6]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class dataset:
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts["input_ids"])

    def __getitem__(self, idx):
        return {k: v[idx] for k, v in self.tokenized_texts.items()}

# Replace 'labels_column' with the actual column name in your dataset containing the labels
def load_test_data(file_name):
    df = pd.read_csv(file_name)
    df = df.sample(frac=1, random_state=42) # Shuffle the data
    split_index = int(0.8 * len(df)) # Splitting into 80-20 train-test split
    train_data = df[:split_index]
    test_data = df[split_index:]
    return test_data['content'].dropna().astype('str').tolist(), test_data['target']

# Loading test data and labels
file_name = "/content/shopee_reviews.csv" # Replace with your file path
test_texts, test_labels = load_test_data(file_name)

# Loading pre-trained model and tokenizer
model_name = "siebert/sentiment-roberta-large-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

# Tokenizing the test texts
tokenized_test_texts = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = dataset(tokenized_test_texts)

# Making predictions on the test dataset
test_predictions = trainer.predict(test_dataset)
test_preds = test_predictions.predictions.argmax(-1)

# Calculating evaluation metrics
test_accuracy = accuracy_score(test_labels, test_preds)
test_precision = precision_score(test_labels, test_preds, average='weighted')
test_recall = recall_score(test_labels, test_preds, average='weighted')
test_f1 = f1_score(test_labels, test_preds, average='weighted')

# Printing the evaluation metrics
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")
print(f"Test F1-score: {test_f1:.4f}")

tokenizer_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Test Accuracy: 0.9102
Test Precision: 0.9103
Test Recall: 0.9102
Test F1-score: 0.9102
